# Zadania

## Zadanie 1: Proste tłumaczenie tekstu za pomocą biblioteki transformers
*Opis*: Użyj modelu tłumaczeniowego z biblioteki Hugging Face `transformers` do przetłumaczenia krótkiego tekstu z języka angielskiego na polski.

## Zadanie 2: Tłumaczenie z detekcją języka źródłowego
*Opis*: Wykryj język źródłowy tekstu za pomocą biblioteki `langdetect`, a następnie przetłumacz go na polski.

## Zadanie 3: Tłumaczenie z detekcją języka źródłowego z HERBERT
*Opis*: Tłumaczenie i Rozszerzanie Tekstu z Wykorzystaniem Modelu HERBERT
Cel zadania:
- Wykryje język wejściowego tekstu.
- Przetłumaczy wykryty tekst na kilkanacie różnych języków.
- Wstawi tokeny <mask> w środkowej części przetłumaczonego zdania (sposób losowy).
- Uzupełni wstawione luki, wykorzystując model HERBERT (np. `allegro/herbert-base-cased`) dostępny przez bibliotekę transformers i jego pipeline `fill-mask`.
- Wykorzytaj biblioteke `googletran` (zalecana wersja: googletrans==4.0.0-rc1)

## Zadanie 4: Tłumaczenie wtórne
*Opis*: Wykryj język źródłowy, przetłumacz na język angielski i polski, dokonaj wielu ponownych tłumaczeń (np. 10 razy z roznych jezyków) i korzystając z `metryki Levenshteina` oceń zdanie początkowe wzgledem tego co wyszło po 10 tłumaczeniach.

## Zadanie 5: Tłumaczenie PDF z użyciem NLP
*Opis*: Wczytaj tekst z pliku PDF, przetłumacz go na polski i zapisz wynik. W tym celu wykorzystaj `PyPDF2`, `RapidOCR` lub `PaddlePaddleOCR`.

## Zadanie 6: Tłumaczenie kontekstowe z pamięcią kontekstu i obsługą dużych dokumentów
*Opis*: Stwórz program, który:  
- Wczytuje duży dokument (np. PDF lub tekstowy) i dzieli go na fragmenty, zachowując kontekst między nimi (np. co zdanie, co 2, co 3).  
- Wykorzystuje model językowy (np. `mBART` lub `T5`) do tłumaczenia z uwzględnieniem kontekstu poprzednich fragmentów.  
- Optymalizuje proces, używając wielowątkowości do równoległego tłumaczenia fragmentów.  
- Zapisuje wynik w formacie `Markdown` z podziałem na sekcje.

## Zadanie 7: Tłumaczenie kontekstowe z pamięcią kontekstu i obsługą dużych dokumentów
*Opis*: Program wczytuje tekst, tłumaczy go z angielskiego na polski za pomocą modelu `Bielik`, pozwala użytkownikowi wybrać ton (np. formalny, potoczny), a wynik zapisuje do pliku tekstowego. Wykorzystaj `Stanza` do wykonania analizy sentymentu. Następnie porównaj rozwiązanie z wykorzystaniem biblioteki `langdetect` i innego transformera, np. `Helsinki-NLP/opus-mt-en-pl`.

## Zadanie 8: Tłumaczenie statystyczne z wykorzystaniem modelu SMT
*Opis*: Stwórz program, który implementuje tłumaczenie statystyczne (Statistical Machine Translation, SMT) z języka angielskiego na polski.  
- Wykorzystaj bibliotekę `Moses` lub `NLTK` do zbudowania modelu SMT na podstawie równoległego korpusu tekstów (np. z Europarl lub OpenSubtitles).  
- Przygotuj dane treningowe, wykonaj tokenizację i wyrównanie słów (word alignment) za pomocą narzędzia `GIZA++`.  
- Przetłumacz krótki tekst testowy i oceń jakość tłumaczenia za pomocą metryki BLEU.  
- Porównaj wyniki z tłumaczeniem opartym na modelu neuronowym (np. `transformers`) pod kątem dokładności i płynności.

In [1]:
!pip install nltk transformers torch sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [7]:
import nltk
from nltk.translate import IBMModel1, AlignedSent, bleu_score
from transformers import MarianTokenizer, MarianMTModel
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')

# 1. Przygotowanie prostego równoległego korpusu (angielski -> polski)
parallel_corpus = [
    AlignedSent(['I', 'like', 'to', 'read', 'books'], ['Lubię', 'czytać', 'książki']),
    AlignedSent(['The', 'cat', 'is', 'on', 'the', 'mat'], ['Kot', 'jest', 'na', 'macie']),
    AlignedSent(['She', 'runs', 'fast'], ['Ona', 'szybko', 'biegnie'])
]

# 2. Trening prostego modelu IBM Model 1 (statystyczne tłumaczenie)
ibm_model = IBMModel1(parallel_corpus, 10)  # 10 iteracji treningu

# 3. Funkcja tłumaczenia statystycznego
def translate_statistical(sentence, model):
    words = word_tokenize(sentence.lower())
    translated = []
    for word in words:
        # Wybierz najbardziej prawdopodobne tłumaczenie dla każdego słowa
        if word in model.translation_table and model.translation_table[word]:
            translated.append(max(model.translation_table[word], key=model.translation_table[word].get))
        else:
            translated.append(word)  # Jeśli brak tłumaczenia, zostaw oryginalne słowo
    return ' '.join(translated)

# 4. Test tłumaczenia statystycznego
test_sentence = "I like to run"
statistical_translation = translate_statistical(test_sentence, ibm_model)
print("Tłumaczenie statystyczne:", statistical_translation)

# 5. Tłumaczenie neuronowe za pomocą MarianMT
model_name = "Helsinki-NLP/opus-mt-en-pl"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

def translate_neural(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True)
    translated = model.generate(**inputs)
    return tokenizer.decode(translated[0], skip_special_tokens=True)

neural_translation = translate_neural(test_sentence)
print("Tłumaczenie neuronowe:", neural_translation)

# 6. Ocena jakości tłumaczenia za pomocą BLEU
reference = ["Lubię biegać"]  # Referencyjne tłumaczenie
statistical_bleu = bleu_score.sentence_bleu([reference], statistical_translation.split())
neural_bleu = bleu_score.sentence_bleu([reference], neural_translation.split())

print(f"BLEU (statystyczne): {statistical_bleu:.4f}")
print(f"BLEU (neuronowe): {neural_bleu:.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Tłumaczenie statystyczne: i Lubię Lubię run


OSError: Helsinki-NLP/opus-mt-en-pl is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [14]:
!pip install langdetect googletrans nest_asyncio sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 15.7 MB/s eta 0:00:00


In [18]:
import nest_asyncio
nest_asyncio.apply()  # Umożliwia ponowne użycie pętli asynchronicznej, np. w Jupyter Notebook

import asyncio
from langdetect import detect
from googletrans import Translator
from transformers import pipeline

def fill_all_masks(sentence, mask_pipeline):
    """
    Funkcja uzupełnia wszystkie wystąpienia tokena <mask> w zdaniu.
    Aby pipeline otrzymał dokładnie jeden token maski, wszystkie wystąpienia poza pierwszym
    zastępujemy tymczasowym placeholderem, który potem przywracamy.
    Uzupełnienie odbywa się sekwencyjnie – zastępujemy pierwszą lukę najlepszą propozycją,
    a następnie przywracamy placeholdery z powrotem do tokenów <mask>.
    """
    placeholder = "UNIQUE_MASK_PLACEHOLDER"
    while "<mask>" in sentence:
        # Podziel zdanie na część przed pierwszym <mask> i resztę
        parts = sentence.split("<mask>", 1)
        # W reszcie zastąp wszystkie pozostałe <mask> placeholderem
        rest = parts[1].replace("<mask>", placeholder)
        # Zbuduj zdanie zawierające tylko jeden token <mask>
        sentence_single_mask = parts[0] + "<mask>" + rest

        # Używamy pipeline'a fill-mask, który oczekuje dokładnie jednego tokena maski
        suggestions = mask_pipeline(sentence_single_mask)
        # Wybieramy najlepszą propozycję (pierwszy element listy)
        best_token = suggestions[0]['token_str']

        # Zamieniamy pierwsze wystąpienie <mask> na najlepszy token
        sentence_filled = sentence_single_mask.replace("<mask>", best_token, 1)
        # Przywracamy placeholder z powrotem do tokenów <mask>
        sentence = sentence_filled.replace(placeholder, "<mask>")
        print("Aktualizowane zdanie:", sentence)
    return sentence

def translate_and_extend(text):
    # 1. Wykrywanie języka
    source_lang = detect(text)
    print(f"Wykryty język: {source_lang}")

    # 2. Tłumaczenie na język polski (używamy asyncio.run dzięki nest_asyncio)
    translator = Translator()
    translation = asyncio.run(translator.translate(text, src=source_lang, dest='pl'))
    translated_text = translation.text
    print("Przetłumaczony tekst:", translated_text)

    # 3. Przygotowanie szablonu zdania z wieloma tokenami <mask>
    extended_sentence_template = translated_text + " <mask> <mask>."
    print("\nSzablon zdania do rozszerzenia:")
    print(extended_sentence_template)

    # 4. Przygotowanie pipeline'a fill-mask z modelem HERBERT
    mask_pipeline = pipeline('fill-mask', model='allegro/herbert-base-cased')

    # 5. Uzupełnienie wszystkich luk
    final_sentence = fill_all_masks(extended_sentence_template, mask_pipeline)
    print("\nFinalne uzupełnione zdanie:")
    print(final_sentence)

    return final_sentence

if __name__ == "__main__":
    # Przykładowy dłuższy tekst wejściowy
    tekst = ("Hello, how are you today? I hope you're having a great day and enjoying every moment!")
    translate_and_extend(tekst)


Wykryty język: en
Przetłumaczony tekst: Witam, jak się masz dzisiaj? Mam nadzieję, że masz wspaniały dzień i cieszysz się każdą chwilą!

Szablon zdania do rozszerzenia:
Witam, jak się masz dzisiaj? Mam nadzieję, że masz wspaniały dzień i cieszysz się każdą chwilą! <mask> <mask>.


Device set to use cpu


Aktualizowane zdanie: Witam, jak się masz dzisiaj? Mam nadzieję, że masz wspaniały dzień i cieszysz się każdą chwilą! @ <mask>.
Aktualizowane zdanie: Witam, jak się masz dzisiaj? Mam nadzieję, że masz wspaniały dzień i cieszysz się każdą chwilą! @ ..

Finalne uzupełnione zdanie:
Witam, jak się masz dzisiaj? Mam nadzieję, że masz wspaniały dzień i cieszysz się każdą chwilą! @ ..


In [20]:
import nest_asyncio
nest_asyncio.apply()  # Pozwala na ponowne użycie pętli asynchronicznej w środowiskach takich jak Jupyter Notebook

import asyncio
from langdetect import detect
from googletrans import Translator
from transformers import pipeline

def insert_masks_in_middle(text, num_masks=2):
    """
    Funkcja wstawia określoną liczbę tokenów <mask> w środkowej części zdania.
    Dzielenie odbywa się na podstawie słów.
    """
    words = text.split()
    mid_index = len(words) // 2
    mask_tokens = " ".join(["<mask>"] * num_masks)
    # Wstawienie tokenów <mask> w środkową część tekstu
    new_words = words[:mid_index] + [mask_tokens] + words[mid_index:]
    return " ".join(new_words)

def fill_all_masks(sentence, mask_pipeline):
    """
    Funkcja sekwencyjnie uzupełnia wszystkie wystąpienia tokena <mask> w zdaniu.
    Aby pipeline działał prawidłowo (oczekuje dokładnie jeden token maski),
    kolejne wystąpienia zastępujemy tymczasowym placeholderem.
    """
    placeholder = "UNIQUE_MASK_PLACEHOLDER"
    while "<mask>" in sentence:
        # Dzielimy zdanie na część przed pierwszym <mask> i resztę
        parts = sentence.split("<mask>", 1)
        # Zastępujemy kolejne wystąpienia tokena <mask> placeholderem
        rest = parts[1].replace("<mask>", placeholder)
        # Odtwarzamy zdanie zawierające dokładnie jeden token <mask>
        sentence_single_mask = parts[0] + "<mask>" + rest

        # Używamy pipeline'a fill-mask, który generuje propozycje dla pojedynczego tokena maski
        suggestions = mask_pipeline(sentence_single_mask)
        # Wybieramy najlepszą propozycję (pierwszą z listy)
        best_token = suggestions[0]['token_str']
        # Zamieniamy pierwsze wystąpienie <mask> na wygenerowany token
        sentence_filled = sentence_single_mask.replace("<mask>", best_token, 1)
        # Przywracamy placeholdery z powrotem do tokenów <mask>, jeśli występują
        sentence = sentence_filled.replace(placeholder, "<mask>")
        print("Aktualizowane zdanie:", sentence)
    return sentence

def translate_and_extend(text):
    # 1. Wykrywanie języka wejściowego
    source_lang = detect(text)
    print(f"Wykryty język: {source_lang}")

    # 2. Tłumaczenie tekstu na język polski
    translator = Translator()
    translation = asyncio.run(translator.translate(text, src=source_lang, dest='en'))
    translated_text = translation.text
    print("Przetłumaczony tekst:", translated_text)

    # 3. Wstawienie tokenów <mask> w środkowej części zdania
    extended_sentence_template = insert_masks_in_middle(translated_text, num_masks=2)
    print("\nSzablon zdania do rozszerzenia:")
    print(extended_sentence_template)

    # 4. Przygotowanie pipeline'a fill-mask z modelem HERBERT
    mask_pipeline = pipeline('fill-mask', model='allegro/herbert-base-cased')

    # 5. Sekwencyjne uzupełnienie wszystkich luk
    final_sentence = fill_all_masks(extended_sentence_template, mask_pipeline)
    print("\nFinalne uzupełnione zdanie:")
    print(final_sentence)

    return final_sentence

if __name__ == "__main__":
    tekst = ("Hello, how are you today? I hope you're having a great day and enjoying every moment!")
    translate_and_extend(tekst)


Wykryty język: en
Przetłumaczony tekst: Hallo, wie geht es dir heute? Ich hoffe, Sie haben einen tollen Tag und genießen jeden Moment!

Szablon zdania do rozszerzenia:
Hallo, wie geht es dir heute? Ich hoffe, <mask> <mask> Sie haben einen tollen Tag und genießen jeden Moment!


Device set to use cpu


Aktualizowane zdanie: Hallo, wie geht es dir heute? Ich hoffe, so <mask> Sie haben einen tollen Tag und genießen jeden Moment!
Aktualizowane zdanie: Hallo, wie geht es dir heute? Ich hoffe, so ! Sie haben einen tollen Tag und genießen jeden Moment!

Finalne uzupełnione zdanie:
Hallo, wie geht es dir heute? Ich hoffe, so ! Sie haben einen tollen Tag und genießen jeden Moment!
